<a href="https://colab.research.google.com/github/cdmurray80/object_detection/blob/main/object_detection1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Docs: https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/2.2.0/install.html#tensorflow-object-detection-api-installation

In [ ]:
# Mount drive
#
#
from google.colab import drive
drive.mount('/content/drive', force_remount=True) 

In [ ]:
import tensorflow as tf
tf.__version__


In [ ]:
def do_setup():
  #
  #%pip install --ignore-installed --upgrade tensorflow
  #
  #!pip install tensorflow==2.8
  !apt install --allow-change-held-packages libcudnn8=8.1.0.77-1+cuda11.2

  import os
  if not os.path.isdir('/content/tensorflow'):
    %mkdir tensorflow
  %cd /content/tensorflow
  !git clone --depth 1 https://github.com/tensorflow/models

  %cd /content/tensorflow/models/research
  %ls
  !protoc object_detection/protos/*.proto --python_out=.

  %pip install cython
  %pip install git+https://github.com/philferriere/cocoapi.git#subdirectory=PythonAPI

  %cd /content/tensorflow/models/research
  !cp object_detection/packages/tf2/setup.py .
  !python -m pip install --use-feature=2020-resolver .

def test_setup():
  !python object_detection/builders/model_builder_tf2_test.py


In [ ]:
do_setup()
test_setup()

In [ ]:
import os
WORKSPACE_DIR='/content/tensorflow/workspace'
test_dir=WORKSPACE_DIR+'/training_demo/images/test'
train_dir=WORKSPACE_DIR+'/training_demo/images/train'
model_dir=WORKSPACE_DIR+'/training_demo/pre-trained-models'
annotations_dir=WORKSPACE_DIR + '/training_demo/annotations'
for d in [WORKSPACE_DIR, WORKSPACE_DIR+'/training_demo', WORKSPACE_DIR+'/training_demo/images', annotations_dir, WORKSPACE_DIR+'/training_demo/scripts', train_dir, test_dir, model_dir]:
  if not os.path.exists(d):
    os.mkdir(d)
    print('Created directory', d)

In [ ]:
#Get all xml files
import glob,shutil
import xml.etree.ElementTree as ET
paths=['/content/drive/MyDrive/Research/object_detection/images/final_MLRS', '/content/drive/MyDrive/Research/object_detection/images/final_t72']
xml_files=[glob.glob(path+'/*.xml') for path in paths]
xml_files=[f for l in xml_files for f in l]

#Move to train and test
for i,f in enumerate(xml_files):
  dir = test_dir if i % 5 == 4 else train_dir
  #Get name of actual image file
  tree=ET.parse(f)
  root=tree.getroot()
  img_file=root[1].text
  f2=f.replace('.xml', '.'+ img_file.split('.')[-1])
  shutil.copy2( f, dir)
  shutil.copy2( f2, dir)

In [ ]:
#Create label file
labels=['MLRS', 'tank']
s=''
for i,l in enumerate(labels):
  s+='''item {\n \t id: %d\n \t name:'%s'\n}\n\n''' % (i+1, l)
#Write file
label_file=annotations_dir+'/label_map.pbtxt'
text_file = open(label_file, 'w')
n = text_file.write(s)
text_file.close()

In [ ]:
#Script to create tfrecords
src = '/content/drive/MyDrive/Research/object_detection/generate_tfrecord.py'
dst = WORKSPACE_DIR+'/training_demo/scripts'
shutil.copy2(src, dst)

In [ ]:
# Create train and test tfrecords
train_record_file=annotations_dir+'/train.record'
test_record_file=annotations_dir+'/test.record'
!python /content/drive/MyDrive/Research/object_detection/generate_tfrecord.py -x $train_dir -l $label_file -o $train_record_file
!python /content/drive/MyDrive/Research/object_detection/generate_tfrecord.py -x $test_dir -l $label_file -o $test_record_file


In [ ]:
import requests

# More models are here
model_name='efficientdet_d1_coco17_tpu-32'
url = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/'+model_name+'.tar.gz'
target_file = WORKSPACE_DIR + '/training_demo/pre-trained-models/'+model_name+'.tar.gz'
response = requests.get(url)
open(target_file, "wb").write(response.content)


In [ ]:
%cd /content/tensorflow/workspace/training_demo/pre-trained-models
!tar -xvzf $target_file 

In [ ]:
#
# The pipeline config points to a few things, including the model, so we'll modify it from a baseline version, then write it
#
with open('/content/drive/MyDrive/Research/object_detection/pipeline.config') as f:
  pipeline_cfg_contents=f.read()
pipeline_cfg_contents=pipeline_cfg_contents.replace('<fine_tune_checkpoint>', 'pre-trained-models/'+model_name + '/checkpoint/')
for outfile in ['/content/drive/MyDrive/Research/object_detection/pipeline_automodified.config', model_dir + '/' + model_name + '/pipeline.config']:
  with open(outfile, 'w') as f:
    f.write(pipeline_cfg_contents)

In [ ]:
shutil.copy2('/content/tensorflow/models/research/object_detection/model_main_tf2.py', WORKSPACE_DIR+'/training_demo')
%cd /content/tensorflow/workspace/training_demo

In [ ]:
%pip install "opencv-python-headless<4.3"

In [ ]:
import tensorflow as tf
tf.__version__

In [ ]:
#!pip install tensorflow==2.7.0
#
# Fixme THIS EARLIER??
#
!ls pre-trained-models/efficientdet_d1_coco17_tpu-32/checkpoint

In [ ]:
#!ls pre-trained-models/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/
!ls pre-trained-models/$model_name/checkpoint

In [44]:
# Actually train!
!python model_main_tf2.py --model_dir=pre-trained-models/$model_name/ --pipeline_config_path=pre-trained-models/$model_name/pipeline.config

2022-06-15 21:15:56.459851: W tensorflow/core/util/tensor_slice_reader.cc:96] Could not open pre-trained-models/efficientdet_d1_coco17_tpu-32/checkpoint/: FAILED_PRECONDITION: pre-trained-models/efficientdet_d1_coco17_tpu-32/checkpoint; Is a directory: perhaps your file is in a different file format and you need to use a different restore operator?
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/training/py_checkpoint_reader.py", line 92, in NewCheckpointReader
    return CheckpointReader(compat.as_bytes(filepattern))
RuntimeError: Unable to open table file pre-trained-models/efficientdet_d1_coco17_tpu-32/checkpoint/: FAILED_PRECONDITION: pre-trained-models/efficientdet_d1_coco17_tpu-32/checkpoint; Is a directory: perhaps your file is in a different file format and you need to use a different restore operator?

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "model_main_tf